# EE 123 - Real Time Sonar App

### Written by Miki Lustig and Frank Ong 2016
#### Updated by Miki Lustig, Li-Hao Yeh, and Michael Chen 2019
##### Updated by Alan Dong, Gautam Gunjala, and Josh Sans 2020

In [1]:
import numpy as np, matplotlib.pyplot as plt
from numpy import *
from scipy import signal
from scipy import interpolate

from rtsonar import rtsonar

Loading BokehJS ...

### Task III: Play with different parameters with the real-time sonar!

Copy and paste your functions `genPulseTrain()`, `genChirpPulse()`, `crossCorr()`, `findDelay()`, and `dist2time()` to the code cell below. Scroll down to the bottom and you should be able to run the real-time sonar!

The detection resolution and sensitivity will depend on the parameters you will choose for the sonar. The longer the pulse is, the worse time resolution you will get, but much stronger matched filtering amplitude. You can get back time resolution by using pulse compression with chirp pulses at the expense of increasing the bandwidth. 

* Run the sonar with different parameters. You can get interesting targets by moving your Pi, moving yourself next to the Pi, using a book as a reflecting surface, etc. It will be easier for you to distinguish the target if you move it back and forth. Play with the pulse lengths, the frequency sweeps, etc. 
* To get the best real-time performance, you should restart the kernel every time you run this lab. 
* Submit 4 interesting sets of parameters. Explain why you chose them.


In [17]:
# Configure the hardware for the experiment
!amixer -c 1 -- sset Lineout playback 100%,100%
!amixer -c 1 -- sset Mic capture 100%
!amixer -c 1 -- sset Mic playback 100%

Simple mixer control 'Lineout',0
  Capabilities: pvolume pswitch pswitch-joined
  Playback channels: Front Left - Front Right
  Limits: Playback 0 - 31
  Mono:
  Front Left: Playback 11 [35%] [-10.00dB] [on]
  Front Right: Playback 11 [35%] [-10.00dB] [on]
Simple mixer control 'Mic',0
  Capabilities: volume volume-joined
  Playback channels: Mono
  Capture channels: Mono
  Limits: 0 - 3
  Mono: 1 [33%] [20.00dB]
Simple mixer control 'Mic',0
  Capabilities: volume volume-joined
  Playback channels: Mono
  Capture channels: Mono
  Limits: 0 - 3
  Mono: 1 [33%] [20.00dB]


In [3]:
# Copy and paste your 5 functions here:

def genChirpPulse(Npulse, f0, f1, fs):
    # Function generates an analytic function of a chirp pulse
    # Inputs:
    #     Npulse - pulse length in samples
    #     f0     - starting frequency of chirp
    #     f1     - end frequency of chirp
    #     fs     - sampling frequency
    # Output:
    #     pulse - chirp pulse
    
    T = Npulse/fs
    t = np.r_[0.0:Npulse]/fs
    k = (f1-f0)/T
    pulse = exp(1j*2*pi*(f0 + k/2*t)*t)
    
    return pulse

def genPulseTrain(pulse, Nrep, Nseg):
    # Function generates a pulse train from a pulse. 
    # Inputs:
    #     pulse  = the pulse generated by genChirpPulse
    #     Nrep   = number of pulse repetitions
    #     Nseg   = length of pulse segment >= len(pulse)
    # Output:
    #     ptrain = pulse train
    
    seg = zeros(Nseg)
    seg[:len(pulse)] = pulse
    ptrain = tile(seg, Nrep)
    
    return ptrain

def crossCorr(rcv, pulse_a):
    # Funtion generates cross-correlation between rcv and pulse_a.
    # Inputs:
    #    rcv - received signal
    #    pulse_a - analytic pulse
    # Output:
    #    Xrcv - cross-correlation between rcv and pulse_a
    
    Xrcv = signal.convolve(rcv, pulse_a[::-1])
    
    return Xrcv

def findDelay(Xrcv, Nseg):
    # Function finds the first pulse index.
    # Inputs:  
    #     Xrcv = the received matched filtered signal
    #     Nseg = length of a segment
    # Output:
    #     ind  = index of the beginning of the first pulse
    
    ind = argmax(Xrcv[0:Nseg])
    
    return ind

def dist2time(dist, temperature=21.0):
    # Function converts distance in cm to time in seconds.
    # Inputs:
    #     dist        = distance to object in cm
    #     temperature = in Celcius
    # Output:
    #     t           = time in seconds between transmitted pulse and echo
    
    v_sound = 331.5*sqrt(1 + temperature/273.15) * 100 # in cm/s
    t = dist/v_sound*2
    
    return t

In [20]:
# Parameter settings
Npulse = 2400
f0     = 123
f1     = 23456
fs     = 48e3
Nseg   = 9600

Nplot       = 250 # number of pixels in plot along horizontal direction (higher resolution is slower)
maxdist     = 250 # maximum distance in cm
maxrep      = 25 # maximum repetition echo shown in the figure
temperature = 20 # temperature
dBf         = 25 # dynamic range of the sonar in dB

functions = (genChirpPulse, genPulseTrain, crossCorr, findDelay, dist2time) # join the functions together

(stop_flag, st) = rtsonar(f0, f1, fs, Npulse, Nseg, maxrep, Nplot, dBf, maxdist, temperature, functions)

In [21]:
# ALWAYS RUN THIS CODE CELL TO STOP THE SONAR BEFORE INTERRUPTING OR RESTARTING THE KERNEL
stop_flag.set()
st.stop()
st.close()